In [ ]:
import joblib
import pandas as pd
import numpy as np

# 1. Chargement du fichier
filename = "lol_champ_model.pkl"
print(f" Chargement de {filename}...")

try:
    data = joblib.load(filename)
    model = data["pipeline"]
    features_needed = data["features"]
    classes = model.classes_
    print(f" Modèle chargé avec succès.")
    print(f" Colonnes attendues ({len(features_needed)}) : {features_needed}")
except FileNotFoundError:
    print(" Fichier introuvable. Vérifiez le chemin.")
    exit()

# 2. Création de 3 profils types pour tester
def create_profile(name, modifications):
    # Dictionnaire de base avec des valeurs moyennes
    profile = {col: 0.0 for col in features_needed} 
    
    # On applique les modifs spécifiques
    for k, v in modifications.items():
        if k in profile:
            profile[k] = v
    return profile

#Jinx : Peu de PV, Beaucoup d'AS et Range 
adc_stats = {
    "HP": 550, "HPperLevel": 90,
    "AttackDamage": 60, "AttackRange": 600,
    "attackspeed": 0.625, "attackspeedperlevel": 3.0,
    "Armor": 25, "Spellblock": 30
}

#Malphite : Énorme PV, Énorme Armure, Pas de range
tank_stats = {
    "HP": 650, "HPperLevel": 110,
    "AttackDamage": 60, "AttackRange": 125, # Corps à corps
    "attackspeed": 0.6, "attackspeedperlevel": 1.5,
    "Armor": 45, "ArmorperLevel": 5.0, # Très haut
    "Spellblock": 32
}

#Lux : Peu de PV, Range moyenne
mage_stats = {
    "HP": 500, "HPperLevel": 80,
    "AttackDamage": 50, "AttackRange": 550,
    "attackspeed": 0.6,
    "Armor": 19, # Très fragile
    "Spellblock": 30
}

tests = [
    ("Simulation ADC", adc_stats),
    ("Simulation TANK", tank_stats),
    ("Simulation MAGE", mage_stats)
]


for nom, stats in tests:
    # Création du DataFrame
    user_input = create_profile(nom, stats)
    X_test = pd.DataFrame([user_input])
    
    # Prédiction
    pred = model.predict(X_test)[0]
    probas = model.predict_proba(X_test)[0]
    
    print(f"\n{nom}")
    print(f"👉 Résultat : {pred}")
    
    # On affiche les détails des probas
    print("📊 Détails :")
    probs_dict = dict(zip(classes, probas))
    # Tri du plus grand au plus petit
    sorted_probs = sorted(probs_dict.items(), key=lambda item: item[1], reverse=True)
    
    for cls, p in sorted_probs[:3]: # Top 3
        print(f"   - {cls}: {p:.1%}")


 Chargement de lol_champ_model.pkl...
 Modèle chargé avec succès.
 Colonnes attendues (17) : ['AttackRange', 'MoveSpeed', 'HP', 'HPperLevel', 'hpregen', 'hpregenperlevel', 'Armor', 'ArmorperLevel', 'AttackDamage', 'AttackDamageperLevel', 'attackspeed', 'attackspeedperlevel', 'Spellblock', 'spellblockperlevel', 'Prv_attack', 'Prv_defense', 'Prv_magic']

Simulation ADC
👉 Résultat : Marksman
📊 Détails :
   - Marksman: 37.0%
   - Support: 32.8%
   - Mage: 15.6%

Simulation TANK
👉 Résultat : Fighter
📊 Détails :
   - Fighter: 28.6%
   - Support: 20.6%
   - Marksman: 17.2%

Simulation MAGE
👉 Résultat : Support
📊 Détails :
   - Support: 51.0%
   - Mage: 25.8%
   - Marksman: 14.6%

